<a href="https://colab.research.google.com/github/jhmartel/fp/blob/master/_notebooks/2022-09-11-WeberThreeBody.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# "[Under Construction] Weber Three Body Problem."
> "Our goal in this post is to integrate the equations of motion for three charged bodies under Weber's force law. Why? Because we want to test the Sansbury proposal that the electron $e^{-}$ is structured, specifically as a 3-body system with net charge -1=-1-1+1. Yes, this is strange hypothesis and very important to investigate. But there is a challenge: the Newtonian equations of motion combined with Weber force are not the usual second order ODEs. This means we need to introduce a Differential Algebraic Equation (DAE) solver named GEKKO, and this has delayed the publication of this post. With GEKKO we can solve ODEs of the form $F(x, x', x'')=0$. This post is theoretical. But we are working on the python notebooks for the actual code for Weber 3-body problem with net charge -1 = -1 -1 + 1. "

- toc: false
- branch: master
- badges: true
- comments: true
- author: JHM
- categories: [fastpages, jupyter]

#__Remark__: 
This article is not yet readable. We've hit a technical difficulty which need to first solve, namely how to integrate ODEs of the form $x''=f(x,x', x'')$ on python. Typically odeint integrates equations of the form $x''=f(x)$ or $x''=f(x,x')$. But the Weber force involves interaction between all the particles their relative accelerations, so applying Newton's 2nd Law gives us equations of motion of the form $$m_i a_i = m_i \frac{dv_i'}{dt}=\sum_j F_{ij}$$ where the force depends on the acceleration of $a_i$ and the particles $a_j$. In the literature this is known as a "Differential Algebraic Equation" (DAE). So we've been looking for python packages which have DAE solvers. It does not appear to be well-known subject.


#__Introduction__: Weber's Force.
Our goal is to study the $N$-body problem on python according to Weber's electrodynamic law. In fact $N=3$ would be good start, and many author's have studied the $2$-body problem, even us in our [first post](https://jhmartel.github.io/fp/fastpages/jupyter/2022/02/10/LetterToAssisWeberPotentials.html) on this blog.

The Weber potential belongs to relational mechanics, being given by a force $F_{12}$ between two charged particles satisfying Newton's action-reaction principle, so $F_{12}=-F_{21}$, and the force is also _radial_ being along the straight line segment joining the centre of the particles. However what is further strange about Weber's potential is that the force $F$ depends on the relative positions, velocity, and accelerations of the particles!

So let us consider a three body system with particles having inertial masses $m_1, m_2, m_3$ and such that they have positions $r_1, r_2, r_3$ relative to the centre of mass $O=O_{CM}$ of the system. We will assume that the calculations are performed in the centre of mass frame.

The total force acting on particle $m_1$ is the sum $F_{12}+F_{13}$, where for example $$F_{1j}=\frac{q_1 q_j}{{r^3}_{1j}} \hat{\bf{r}}_{{1j}}  \left( 1-\frac{{r'_{1j}}^2}{2 c^2} + \frac{r_{1j} r_{1j}''}{c^2}    \right) $$ for $j=2,3$.

Now if we directly apply Newton's 2nd Law we get:
$$F_{\text{net} 1}=F_{21}+F_{31}=m_1 \frac{dv_1}{dt}.$$ Here we want to be extremely careful in interpreting these terms. The velocity $v_1$ is the velocity of the particle $m_1$ with respect to the centre-of-mass, i.e. we have $v_1=v_{1O}$. The difficulty with the second order differential equation obtained from Newton's 2nd Law is that $r_{12}''$ is an acceleration term that has some component in common with $\frac{dv_1}{dt}$. The term $r_{12}''$ is very different from $r_1$. Indeed we view $r_1$ and $dr_1/dt=v_1$ as vectors (a position and velocity vector). However the radial $r_{12}$ and $r_{12}'$, $r_{12}''$ are _scalar-valued_ functions!

To use python's odeint solver, we either need to rewrite the expression defining Newton's 2nd Law, or we need to directly define an integration method which integrates the equation _as is_. 

What are the terms that enter into the ODE? 

We have the states $state[i]$ for $i=1,2,3$. Each state has position, velocity, and acceleration relative to the centre-of-mass $O$:

$$state[i]=[r_i, v_i, a_i, q_i, m_i].$$ Here $q_i$ represents the electric charge, and $m_i$ the inertial mass.

The force on $state[i]$ caused by the other states $state[j]$ depends on the relative _radial_ positions, velocities, and accelerations,  namely via:

$$r_{ij}:=|r_i - r_j|  $$ and 
$$v_{ij}:=v_i - v_j  $$ and 
$$a_{ij}:=a_i -a_j $$ and
$$r'_{ij}:= {\hat{\bf{r}}}_{ij} \cdot v_{ij}  $$ and 
$$r''_{ij}:=\frac{ {v_{ij} \cdot v_{ij}} - ({\hat{r}}_{ij} \cdot v_{ij})^2 +r_{ij} \cdot a_{ij}} {r_{ij}}.$$

This last identity for $r''_{ij}$ is very important for our calculations. If we directly substitute this formula into Weber's force law and Newton's 2nd Law we get:

$$m_i a_i= f_i(r_{ij}, v_{ij}) + \sum_{j} \frac{q_i q_j}{r_{ij}^2 c^2} ~ {\hat{\bf{r}}}_{ij}~ ( r_{ij} \cdot a_{ij}), $$ where $f_i$ is defined as 

$$f_i(r_{ij}, v_{ij}):=\sum_j \frac{q_i q_j}{r^2_{ij}} \hat{\bf{r}}_{ij} \left( 1-\frac{{r'_{ij}}^2}{2 c^2} + \frac{1}{c^2} [ v_{ij}\cdot v_{ij} -  ({\hat{\bf{r}}}_{ij} \cdot v_{ij})^2      ]\right) $$ for $1\leq i , j \leq 3$.

Using the above formulas we rewrite $r'_{ij} = \hat{\bf{r}}_{ij} \cdot v_{ij}$, and therefore everything in $f_i$ can be computed directly from $ \{ {\hat{\bf{r}}}_{ij}\}$ and $\{v_{ij}\}$. 


These are all the terms that enter into the calculation of Weber's force law. It's more complicated than Newton's Gravitational equations, which depends only on the relative positions $\{r_{ij}\}$. Notice that $r_{ij}$, $r'_{ij}$, $r''_{ij}$ are scalar-valued, and the terms $r_i$, $v_i$, $v_{ij}$, and $a_{ij}$ are vector-valued. 

Now the auxiliary function $f_i$ is ready-made for python's odeint. But the second right hand term is more interesting, as it involves the relative acceleration term directly, where $a_{ij}:=a_i - a_j$ by definition. We are looking to solve for $a_i$, for all $i$. The challenge is to: 
1. decouple the equations into independant ODEs
2. try to rearrange the terms, see what symbolic manipulation can yield. This means writing it out in full (or in Wolfram Mathematica) and staring at it for a good while. Time for experimentation with the symbols.

Remark. An initial approximation might involve ignoring this second term. In fact we should rewrite our python code to reflect this direct factorization. Note that $r'_{ij}$ is still present in our formula, but we could directly use formula $r'_{ij}=\hat{\bf{r}}_{ij} \cdot v_{ij}$.



In [ ]:
# hide

import numpy as np
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

# We begin with defining the force vectors on a given state of N-particles.
# The print statements are for testing purposes. They can be ignored.

def WeberForce(state):
    state_array = np.asarray(state, dtype=object)
    N=state_array.shape[0]
    #N=2
    #print(N)
    F_net=np.zeros((N,3))
    
#    print(type(F_net))

    c=1.0 #
    
    for i in range(N):
        for j in range(N):
            r_ij=np.subtract(state[i][0:3], state[j][0:3])
#            print("r_ij is equal to", r_ij)
            rho_ij=np.dot(r_ij, r_ij)**(0.5)
#            print("rho_ij is equal to", rho_ij)
            rhat_ij=np.dot(r_ij, 1/rho_ij)
#            print("rhat_ij is equal to", rhat_ij, "and has type", type(rhat_ij))
            v_ij=np.subtract(state[i][3:6],state[j][3:6])
#            print("v_ij is equal to", v_ij)
            dr_ij=np.dot(rhat_ij, v_ij)
#            print("r'_ij is equal to", dr_ij)
            A=np.dot(v_ij, v_ij)
#            print("The A term is equal to:", A)
            a_ij = np.subtract(state[i][6:9], state[j][6:9])
#            print("The a_ij term is:", a_ij)
            B=dr_ij
#            print("The B term is equal to:", B)
            C=np.dot(r_ij, a_ij)
#            print("The C terms are equal to:", C)
            factor1=state[i][9]*state[j][9]*(rho_ij**-2)  ## product of the charges q_i * q_j and rsquared denominator.
            factor2=factor1 * ( 1 - ((dr_ij/c)**2)/2 + (A - B**2)*(c**-2)       )
            factor3=factor1 * ( (np.dot(r_ij, a_ij))*(c**-2)  )

            vector1_ij = np.dot(factor2, rhat_ij)
            vector2_ij = np.dot(factor3, rhat_ij)


            if i!=j:
               F_net[i] = F_net[i]+vector1_ij+vector2_ij
            else:
               pass

    return F_net

## We have decomposed the force into the sum described above.
## Again the problem is to isolate for a_i directly.


In [ ]:
# hide

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
#warnings.filterwarnings("ignore", category=VisibleDeprecationWarning)  
## We test the WeberForce function on a test state
state_test= [[0,1,1, 0,0,0, 1,0,0, -1, 1.0], [1,0,1, 0,0,0, 0,1,0, -1, 1.0], [1,1,0, 0,0,0, 0,0,-1, +1, 500.0]]
# We see the test state consists of three particles at different positions, with different relative velocities, 
# and charges.

#output 
force_test=WeberForce(state_test)
print(force_test)
print(force_test.shape)


In [ ]:
# hide

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# We slightly modify the WeberForce function to get direct acceleration vector on each
# particle. Now here there is slight possibility of confusion: In our setting, the particles have
# an initial acceleration. According to Weber's force law, the particles in their motions will also
# experience acceleration caused by the Weber force. This resultant acceleration will *modify* the 
# initial acceleration of the particles and changing their trajectories.

def Acceleration(state):
    state_array = np.asarray(state, dtype=object)
    N=state_array.shape[0]
    a=np.zeros((N,3))
    for i in range(N):
        a[i] = np.dot(WeberForce(state)[i], 1/state[i][10] )
 # Note: we have not yet divided by the inertial mass of the object m_i.  
    return a

Acceleration(state_test)

array([[ 0.70710678, -0.35355339, -0.35355339],
       [-0.35355339,  0.70710678, -0.35355339],
       [-0.00070711, -0.00070711,  0.00141421]])

In [ ]:
# hide

for i in range(N):
        for j in range(N):
            r_ij=np.subtract(state[i][0:3], state[j][0:3])
#            print("r_ij is equal to", r_ij)
            rho_ij=np.dot(r_ij, r_ij)**(0.5)
#            print("rho_ij is equal to", rho_ij)
            rhat_ij=np.dot(r_ij, 1/rho_ij)
#            print("rhat_ij is equal to", rhat_ij, "and has type", type(rhat_ij))
            v_ij=np.subtract(state[i][3:6],state[j][3:6])
#            print("v_ij is equal to", v_ij)
            dr_ij=np.dot(rhat_ij, v_ij)
#            print("r'_ij is equal to", dr_ij)
            A=np.dot(v_ij, v_ij)
#            print("The A term is equal to:", A)
            a_ij = np.subtract(state[i][6:9], state[j][6:9])
#            print("The a_ij term is:", a_ij)
            B=dr_ij
#            print("The B term is equal to:", B)
            C=np.dot(r_ij, a_ij)
#            print("The C terms are equal to:", C)
            factor1=state[i][9]*state[j][9]*(rho_ij**-2)  ## product of the charges q_i * q_j and rsquared denominator.
            factor2=factor1 * ( 1 - ((dr_ij/c)**2)/2 + (A - B**2)*(c**-2)       )
            factor3=factor1 * ( (np.dot(r_ij, a_ij))*(c**-2)  )

            vector1_ij = np.dot(factor2, rhat_ij)
            vector2_ij = np.dot(factor3, rhat_ij)
